# 8.1 卷积(convnets) 介绍

惯例,在了解 cnn 之前,先上例子.

还记得最开始 mnist 的例子吗? 2 层的全连接层的准确率在 98.22%.但是哪怕只是最简单的堆砌两层 cnn 模型的准确度就能超过第一个例子.


In [13]:
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
inputs = keras.Input(shape=(28, 28, 1))  #输入大小
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)  #卷积层
x = layers.MaxPooling2D(pool_size=2)(x)  #池化层
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)  #卷积层
x = layers.MaxPooling2D(pool_size=2)(x)  #
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)  #卷积层
x = layers.Flatten()(x)  #展平
outputs = layers.Dense(10, activation="softmax")(x)  #输出层
model = keras.Model(inputs=inputs, outputs=outputs)  #模型

传递给第一个 cnn 的大小是 28x28x1 这是 mnist 数据的大小


In [15]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0   

model

- 每个 conv2d 和 max_pooling2d 输出都是 3d 张量.
- conv2d 和 max_pooling2d 的输出尺寸在逐层变小,但是另一个维度在指数增长.
- cnn 最后的输出是 3x3x128 ,后面接了 Flatten 将 3d 张量变成了 1d 张量.
- 最后一层是全连接层,输出 1d 标量.


In [16]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

数据


In [17]:
model.compile(
    optimizer="rmsprop",  #优化器
    loss="sparse_categorical_crossentropy",  #损失函数
    metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 6s 6ms/step - loss: 0.1562 - accuracy: 0.9515
Epoch 2/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0444 - accuracy: 0.9864
Epoch 3/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0310 - accuracy: 0.9902
Epoch 4/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0236 - accuracy: 0.9929
Epoch 5/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0189 - accuracy: 0.9947


基本和第二章例子保持一致.

- 十路的分类,最后是 softmax 激活,使用交叉熵验证损失.


In [18]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0279 - accuracy: 0.9914
Test accuracy: 0.991


只将 layer 替换成 cnn 效果一次就到了 99.1%,还是让我有些惊讶.


## 卷积操作本质

卷积层和全连接层最本质的区别是: 全连接层从输入数据的学习到的是全局模式,而卷积层学到的是局部模式.

如下图所示,全连接层是一次看到整个图片学习,卷积层是一次只看 3x3 区域学习.

![local_patterns.jpg](./local_patterns.jpg)


这让 cnn 具有了两个非常有趣的性质

- cnn 学习到的模型具有平移不变性: 说人话就是假设 cnn 从 3x3 学习到了模式,当这个 3x3 又在其他图片重复出现时,这个模式也通用,不再需要重复训练,因为 cnn 见过了,已经记住了.但是全连接层不行,全连接层一次看整张图片,即使有一个很相似的新图片出现,全连接层也只能重新训练.这个性质使得训练需要的样本数量更少了.
- cnn 可以学习到模式的空间层次结构: 书上说的有些抽象.前面的例子中 cnn 一层一层的堆叠,不停的拆解局部特征.这个过程和人识别物体很像(学术上叫 视觉世界具有空间层次结构)人看到猫 -> 人认知到这是猫.![visual_hierarchy_hires](./visual_hierarchy_hires.png)


卷积操作在 3d 张量上操作,这样的张量叫做特征图,有两个空间轴(宽度高度)还有一个深度轴(也叫通道轴).对于 RGB 图像,深度轴是3维.灰度图像,深度轴是 1 维.卷积运算从输入的特征图提取图块,对这些图块进行相同的变换,生成**输出特征图**.

输出特征图依旧是一个 3d 张量,宽度高度和深度.唯独深度是任意的,这个由卷积层的参数决定.经过一层卷积层,深度轴不再代表原始的图像信息,而是卷积层对图像的过滤结果-过滤器.单个过滤器代表的是从图像提取的特征(模式).

mnist 的例子中,第一个 cnn 接收 28x28x1 的图像,输出 26x26x32 的特征图.这一层有计算 32 个过滤器,32 个输出通道,每个通道上都包含了 26x26 的数值网络.它是过滤器对输入的响应图,表明这个过滤器在输入不同位置的响应.

深度轴每个维度都是特征(模式/过滤器),而 2d `output[:,:,n]` 代表的是过滤器在输入图像上的二维响应图.(表明这个特征/模式/过滤器 在原始图像上这个位置,存在的可能性大小分布)

![response_map_hires](./response_map_hires.png)


卷积有两个重要的关键参数

- 从输入中提取的图块的尺寸.一般选择有 3x3 5x5 或者 7x7.
- 提取特征图的深度,卷积层的过滤器数量.这个例子中有 32 64.
